In [3]:
import time
import numpy as no
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

In [5]:
## Settings

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


## hyperparameters
random_seed = 1
learning_rate = 0.1
num_epochs = 10
batch_size = 64

## Architecture
num_features = 784
num_hidden_1 = 128
num_hidden_2 = 256
num_classes = 10

In [6]:
## MNIST dataset

train_dataset = datasets.MNIST(root='../data',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)

test_dataset = datasets.MNIST(root='../data',
                              train=False,
                              transform=transforms.ToTensor(),
                              download=True)

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size,
                         shuffle=False)

# checking
for images, labels in train_loader:
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw

Image batch dimensions: torch.Size([64, 1, 28, 28])
Image label dimensions: torch.Size([64])


In [10]:
## Model

class MultilayerPerceptron(torch.nn.Module):

    def __init__(self, num_features, num_classes):
        super(MultilayerPerceptron, self).__init__()

        # 1st hidden layer
        self.linear_1 = torch.nn.Linear(num_features, num_hidden_1)
        self.linear_1.weight.detach().normal_(0.0, 0.1)
        self.linear_1.bias.detach().zero_()
    
        # 2nd hidden layer
        self.linear_2 = torch.nn.Linear(num_hidden_1, num_hidden_2)
        self.linear_2.weight.detach().normal_(0.0, 0.1)
        self.linear_1.bias.detach().zero_()
    
        # outout layer                                
        self.linear_out = torch.nn.Linear(num_hidden_2, num_classes)
        self.linear_out.weight.detach().normal_(0.0, 0.1)
        self.linear_out.bias.detach().zero_()

    def forward(self, x):
        out = self.linear_1(x)
        out = F.relu(out)
        out = self.linear_2(out)
        out = F.relu(out)
        logits = self.linear_out(out)
        probas = F.log_softmax(logits, dim=1)
        return logits, probas

torch.manual_seed(random_seed)
model = MultilayerPerceptron(num_features=num_features,
                             num_classes=num_classes)

model = model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [13]:
def compute_accuracy(net, data_loader):
    net.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.view(-1, 28*28).to(device)
            targets = targets.to(device)
            logits, probas = net(features)
            _, predicted_labels = torch.max(probas, 1)
            num_examples += targets.size(0
            correct_pred += (predicted_labels == targets).sum()
        return correct_pred.float() / num_examples *100

start_time = time.time()
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):

        features = features.view(-1, 28*28).to(device)
        targets = targets.to(device)

        ## forward and back prop
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()

        cost.backward()

        ## update model params
        optimizer.step()

        ## logging
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_loader), cost))

    with torch.set_grad_enabled(False):
        print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
              epoch+1, num_epochs, 
              compute_accuracy(model, train_loader)))
        
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/010 | Batch 000/938 | Cost: 0.0909
Epoch: 001/010 | Batch 050/938 | Cost: 0.1402
Epoch: 001/010 | Batch 100/938 | Cost: 0.1861
Epoch: 001/010 | Batch 150/938 | Cost: 0.3434
Epoch: 001/010 | Batch 200/938 | Cost: 0.0945
Epoch: 001/010 | Batch 250/938 | Cost: 0.1481
Epoch: 001/010 | Batch 300/938 | Cost: 0.0442
Epoch: 001/010 | Batch 350/938 | Cost: 0.1991
Epoch: 001/010 | Batch 400/938 | Cost: 0.0805
Epoch: 001/010 | Batch 450/938 | Cost: 0.0864
Epoch: 001/010 | Batch 500/938 | Cost: 0.1280
Epoch: 001/010 | Batch 550/938 | Cost: 0.1653
Epoch: 001/010 | Batch 600/938 | Cost: 0.1616
Epoch: 001/010 | Batch 650/938 | Cost: 0.3182
Epoch: 001/010 | Batch 700/938 | Cost: 0.2119
Epoch: 001/010 | Batch 750/938 | Cost: 0.0994
Epoch: 001/010 | Batch 800/938 | Cost: 0.1729
Epoch: 001/010 | Batch 850/938 | Cost: 0.0441
Epoch: 001/010 | Batch 900/938 | Cost: 0.2079


TypeError: unsupported operand type(s) for +=: 'int' and 'torch.return_types.max'